In [92]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
from mesa import Agent, Model
from mesa.time import RandomActivation

In [77]:
class set_up_t_agents():
    t_agents = generate_t_agents()
    pred_t_agents = generate_pred_agents()

    def generate_t_agents():
        agents = pd.DataFrame()

        ages = np.round(np.random.normal(36, 5, 10000), 0)
        males = np.zeros((9000,), dtype=int)
        females = np.ones((1000,), dtype=int)
        genders = np.concatenate((males, females), axis=None)
        reli = np.zeros((8000,), dtype=int)
        relo = np.ones((2000,), dtype=int)
        religions = np.concatenate((reli, relo), axis=None)
        #fighter = np.zeros((5000,), dtype=int)
        #funding = np.ones((2200,), dtype=int)
        #intel = np.full((1,2200), 2)
        #command = np.full((1,600), 3)
        #roles = np.concatenate((fighter, funding, intel, command), axis=None)
        np.random.shuffle(genders)
        np.random.shuffle(religions)
        #np.random.shuffle(roles)
        #agr_bhv = np.random.ranf((10000,))
        X1 = get_truncated_normal(mean=.25, sd=.25, low=0, upp=1)
        X2 = get_truncated_normal(mean=.5, sd=.15, low=0, upp=1)
        X3 = get_truncated_normal(mean=.35, sd=.3, low=0, upp=1)
        X4 = get_truncated_normal(mean=.2, sd=.35, low=0, upp=1)
        X5 = get_truncated_normal(mean=.5, sd=.4, low=0, upp=1)
        agr_bhv = X1.rvs(10000)
        rel_fnt = X2.rvs(10000)
        hst_twd_for = X3.rvs(10000)
        lvl_rct_act = X4.rvs(10000)
        crt_agr_lvl = X5.rvs(10000)

        agents['ages'] = ages.astype(int)
        agents['gender'] = genders
        agents['religion'] = religions
        #agents['role'] = roles
        agents['agr_bhv'] = agr_bhv
        agents['rel_fnt'] = rel_fnt
        agents['hst_twd_for'] = hst_twd_for
        agents['lvl_rct_act'] = lvl_rct_act
        agents['crt_agr_lvl'] = crt_agr_lvl

        return agents

    def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
        return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

    def generate_pred_agents():
        agents = pd.DataFrame()

        ages = np.round(np.random.normal(36, 5, 10000), 0)
        males = np.zeros((9000,), dtype=int)
        females = np.ones((1000,), dtype=int)
        genders = np.concatenate((males, females), axis=None)
        reli = np.zeros((8000,), dtype=int)
        relo = np.ones((2000,), dtype=int)
        religions = np.concatenate((reli, relo), axis=None)
        #fighter = np.zeros((5000,), dtype=int)
        #funding = np.ones((2200,), dtype=int)
        #intel = np.full((1,2200), 2)
        #command = np.full((1,600), 3)
        #roles = np.concatenate((fighter, funding, intel, command), axis=None)
        np.random.shuffle(genders)
        np.random.shuffle(religions)
        #np.random.shuffle(roles)
        #agr_bhv = np.random.ranf((10000,))
        X1 = get_truncated_normal(mean=.25, sd=.25, low=0, upp=1)
        X2 = get_truncated_normal(mean=.5, sd=.15, low=0, upp=1)
        X3 = get_truncated_normal(mean=.35, sd=.3, low=0, upp=1)
        X4 = get_truncated_normal(mean=.2, sd=.35, low=0, upp=1)
        X5 = get_truncated_normal(mean=.5, sd=.4, low=0, upp=1)
        agr_bhv = X1.rvs(10000)
        rel_fnt = X2.rvs(10000)
        hst_twd_for = X3.rvs(10000)
        lvl_rct_act = X4.rvs(10000)
        crt_agr_lvl = X5.rvs(10000)

        agents['ages'] = ages.astype(int)
        agents['gender'] = genders
        agents['religion'] = religions
        #agents['role'] = roles
        agents['agr_bhv'] = agr_bhv
        agents['rel_fnt'] = rel_fnt
        agents['hst_twd_for'] = hst_twd_for
        agents['lvl_rct_act'] = lvl_rct_act
        agents['crt_agr_lvl'] = crt_agr_lvl

        agents['is_current_threat'] = None
        agents.loc[(agents['lvl_rct_act'] >= 0.75) | ((agents['rel_fnt'] >= 0.75) & (agents['hst_twd_for'] >= 0.75) & (agents['crt_agr_lvl'] >= 0.6)), 'is_current_threat'] = 1
        agents.loc[(agents['lvl_rct_act'] < 0.75) | ((agents['rel_fnt'] < 0.75) & (agents['hst_twd_for'] < 0.75) & (agents['crt_agr_lvl'] < 0.6)), 'is_current_threat'] = 0

        return agents

In [111]:
class TerroristAgent(Agent):
    
    def __init__(self, unique_id, model, agent, pred_model):
        super().__init__(unique_id, model)
        self.age = int(agent.ages)
        self.gender = int(agent.gender)
        self.religion = int(agent.religion)
        self.agr_bhv = float(agent.agr_bhv)
        self.rel_fnt = float(agent.rel_fnt)
        self.hst_twd_for = float(agent.hst_twd_for)
        self.lvl_rct_act = float(agent.lvl_rct_act)
        self.crt_agr_lvl = float(agent.crt_agr_lvl)
        self.prob_threat = 0
        
    def step(self):
        self.prob_threat = pred_model.predict(self)

In [115]:
class MapModel(Model):
    
    pred_model = None
    
    def __init__(self):
        set_up =  set_up_t_agents()
        t_agents = set_up.t_agents
        pred_agents = set_up.pred_t_agents
        pred_model = train_model(pred_agents)
        self.schedule = RandomActivation(self)
        for x in range(len(t_agents)):
            a = TerroristAgent(x, self, t_agents[x:x+1], pred_model)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()
        
    def train_model(self, pred_agents):
        params = {'loss': 'ls', 'learning_rate': .1, 'n_estimators': 500, 'max_depth': 10}
        regr = ensemble.GradientBoostingRegressor(**params)
        X = pred_agents.drop()
        regr.fit()
        
        return regr

In [113]:
empty_model = MapModel()

In [114]:
empty_model.step()

BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
BOOM
